# Predicting High-Usage Players on the Minecraft Server


In [1]:
library(tidyverse)
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimite

## 1. Data Description

In [9]:
summary(players)
summary(sessions)
head(players, 5)
head(sessions, 5)

  experience        subscribe       hashedEmail         played_hours    
 Length:196         Mode :logical   Length:196         Min.   :  0.000  
 Class :character   FALSE:52        Class :character   1st Qu.:  0.000  
 Mode  :character   TRUE :144       Mode  :character   Median :  0.100  
                                                       Mean   :  5.846  
                                                       3rd Qu.:  0.600  
                                                       Max.   :223.100  
                                                                        
     name              gender               Age       
 Length:196         Length:196         Min.   : 9.00  
 Class :character   Class :character   1st Qu.:17.00  
 Mode  :character   Mode  :character   Median :19.00  
                                       Mean   :21.14  
                                       3rd Qu.:22.75  
                                       Max.   :58.00  
                               

 hashedEmail         start_time          end_time         original_start_time
 Length:1535        Length:1535        Length:1535        Min.   :1.712e+12  
 Class :character   Class :character   Class :character   1st Qu.:1.716e+12  
 Mode  :character   Mode  :character   Mode  :character   Median :1.719e+12  
                                                          Mean   :1.719e+12  
                                                          3rd Qu.:1.722e+12  
                                                          Max.   :1.727e+12  
                                                                             
 original_end_time  
 Min.   :1.712e+12  
 1st Qu.:1.716e+12  
 Median :1.719e+12  
 Mean   :1.719e+12  
 3rd Qu.:1.722e+12  
 Max.   :1.727e+12  
 NA's   :2          

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12


In [10]:
# Count how many unique players appear in sessions.csv
n_sessions_players <- sessions |> distinct(hashedEmail) |> nrow()

# Count total players in players.csv
n_total_players <- nrow(players)

# Print both results
n_sessions_players
n_total_players

[1] 125

[1] 196

The project uses two datasets collected from the UBC Minecraft research server:  
**`players.csv`** and **`sessions.csv`**.

#### players.csv
- **Observations:** 196 unique players  
- **Variables:** 7  
- **Description of variables:**

| Variable | Type | Description | Example | Issues / Notes |
|-----------|------|--------------|----------|----------------|
| `experience` | Categorical (character) | Player’s self-reported Minecraft experience level | "Veteran", "Pro", "Amateur" | Uneven group sizes possible |
| `subscribe` | Logical (TRUE/FALSE) | Whether the player subscribed to the research newsletter | TRUE | Could serve as a target variable for Question 1 |
| `hashedEmail` | Character | Unique player identifier (used to link to sessions.csv) | long hash string | Acts as join key |
| `played_hours` | Numeric | Total reported playtime (hours) | Mean = 5.85, Max = 223.1 | Strong right skew — some players play far more than others |
| `name` | Character | Player’s name | “Morgan” | Not useful for prediction |
| `gender` | Categorical | Player’s gender | “Male”, “Female” | May contain small or unbalanced categories |
| `Age` | Numeric | Player’s age (years) | Mean = 21.14, Range = 9–58 | 2 missing values |

---
#### sessions.csv
- **Observations:** 1,535 individual play sessions  
- **Variables:** 5  
- **Description of variables:**

| Variable | Type | Description | Example | Issues / Notes |
|-----------|------|--------------|----------|----------------|
| `hashedEmail` | Character | Player identifier linking to players.csv | same hash as above | Used for join |
| `start_time` | Character (datetime) | Start time of session (`dd/mm/yyyy hh:mm`) | 30/06/2024 18:12 | Needs parsing to datetime |
| `end_time` | Character (datetime) | End time of session (`dd/mm/yyyy hh:mm`) | 30/06/2024 18:24 | Needs parsing to datetime |
| `original_start_time` | Numeric (timestamp) | Unix-like start time (ms) | 1.719e+12 | Duplicate info, potential conversion issue |
| `original_end_time` | Numeric (timestamp) | Unix-like end time (ms) | 1.719e+12 | Often equal to start, possible recording bug |

---

#### Potential Issues
- 71 players have no sessions (appear only in players.csv).  
- `played_hours` and session duration both measure playtime which may overlap.  
- The time format (`dd/mm/yyyy`) must be converted for time calculations.  
- `original_*` timestamps seem identical, might be just recording the date.  
- Possible data imbalance: more players with low playtime than high playtime.  
- Sampling bias may exist; data collected from voluntary participants.

---

#### Summary Statistics (rounded to 2 decimals)
- Mean player age: **21.14 years**
- Mean reported playtime: **5.85 hours**
- Median playtime: **0.10 hours**
- Max playtime: **223.10 hours**

These statistics suggest that most players spend little time on the server, but a few players contribute a lot more data which may bias predictions to those of high-usage players.


## 2. Questions

## 3. Exploratory Data Analysis & Visualization

## 4. Methods and Plan

## 5. GitHub Repository